In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [5]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [7]:
ls data

men_shoes.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [9]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
def run_model(feats,model = DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model,X,y,scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [19]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [20]:
model = RandomForestRegressor(max_depth=5,n_estimators=100,random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [23]:
 df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [29]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

literal_eval(str_dict)
literal_eval(str_dict)[0]
literal_eval(str_dict)[0]['value']

['Men']

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan':return output_dict

  features =  literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [45]:
keys = set()


df['features_parsed'].map(lambda x: keys.update(x.keys()))

len(keys)

476

In [47]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats:feats[key] if key in feats else np.nan)


In [48]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_manufacturer', 'feat_elastic banded waist',
       'feat_location - country', 'feat_polarized lenses', 'feat_name',
       'feat_looks best on these face shapes:', 'feat_condition',
       'feat_mirrored', 'feat_heart rate monitor', 'feat_style name'],
      dtype='object', length=526)

In [50]:
df[df['feat_athlete'].isnull() ].shape

(18272, 526)

In [51]:
df.shape[0]

18280

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[False ==df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] *100

In [60]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key + '_cat')] = df[get_name_feat(key)].factorize()[0]

In [93]:
df['brand'] = df['brand'].map(lambda x:str(x).lower())
df[df.brand == df.feat_brand][['brand','feat_brand']].head()

,brand,feat_brand
0,josmo,josmo
1,josmo,josmo
2,servus by honeywell,servus by honeywell
3,servus by honeywell,servus by honeywell
4,servus by honeywell,servus by honeywell


In [85]:
df[df.brand != df.feat_brand].shape

(9434, 533)

In [86]:
df[df.brand == df.feat_brand].shape

(8846, 533)

In [96]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_location - city/state',
 'feat_clothing category',
 'feat_catalog',
 'feat_fabrication',
 'feat_shoe category',
 'feat_multi pack indicator',
 'feat_recommended location',
 'feat_certifications and listings',
 'feat_location - country',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_band manufacturer_cat',
 'feat_stone type_cat',
 'feat_has paper wood_cat',
 'feat_country of origin_cat',
 'feat_uv rating_cat',
 'feat_international shipping?_cat',
 'feat_power reserve_cat',
 'feat_compass_cat',
 'feat_shoe fastening type_cat',
 'feat_purpose_cat',
 'feat_band material_cat',
 'feat_clothing size_cat',
 'feat_ground_cat',
 'feat_waist_cat',
 'feat_type 2_cat',
 'feat_includes_cat',
 'feat_case finish_cat',
 'feat_fabric type_cat',
 'feat_manufacturer sku_cat',
 'feat_country of manufacture_cat',
 'feat_wheel type_cat',
 'feat_adidas_cat',


In [0]:
feats = ['brand_cat','feat_brand_cat','feat_gender_cat','feat_material_cat', 'feat_sport_cat','feat_style_cat', 'feat_metal type_cat', 'feat_shape_cat']
#feats += feats_cat
#feats = list(set(feats))


In [119]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats,model)

(-57.212390176155836, 4.204021078221484)

In [120]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

perm = PermutationImportance(m,random_state=1).fit(X,y);
eli5.show_weights(perm,feature_names=feats)

Weight,Feature
0.2566 ± 0.0079,brand_cat
0.1034 ± 0.0163,feat_material_cat
0.0241 ± 0.0038,feat_gender_cat
0.0181 ± 0.0010,feat_brand_cat
0.0128 ± 0.0003,feat_shape_cat
0.0085 ± 0.0010,feat_metal type_cat
0.0035 ± 0.0008,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [82]:
df['brand'].value_counts(normalize=True)

nike                   0.097210
puma                   0.033315
ralph lauren           0.028775
vans                   0.021116
new balance            0.020295
                         ...   
vionic by orthaheel    0.000055
rugged blue            0.000055
kingshow               0.000055
nba                    0.000055
korkers                0.000055
Name: brand, Length: 1732, dtype: float64

In [83]:
df[df['brand'] == 'nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)